In [2]:
# 导入pandas和numpy库
import pandas as pd
import numpy as np
# 读取数据集
user_artists = pd.read_csv('./music/user_artists.dat', sep='\t') # 优用"t'分隔符读取user artists.dat文件
artists = pd.read_csv('./music/artists.dat', sep='\t', usecols=['id', 'name']) # 使用t分隔符读取artists.dat文件，仅读取id和name列
user_artists

# 去除不必要的列
# user_artists = user_artists.drop(['weight'], axis=1)# 删除user _artists中的weight列

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


In [3]:
artists.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


In [5]:
artists['id'] =artists['id'].astype(np.int64)

6.将user_artists和artists两个DataFrame进行合并，以user_artists中的artistID和artists中的id为键值进行连接。

In [6]:
# 合并数据集
data = pd.merge(user_artists, artists, left_on='artistID', right_on='id') # 将user_artists 和 artists 合并
data = data.drop(['id', 'artistID'], axis=1) # 删除data 中的id 和artistID 列
data

,userID,weight,name
0,2,13883,Duran Duran
1,2,11690,Morcheeba
2,2,11351,Air
3,2,10300,Hooverphonic
4,2,8983,Kylie Minogue
...,...,...,...
92829,2100,337,Nyktalgia
92830,2100,297,Atsakau niekadA
92831,2100,281,Domantas Razauskas
92832,2100,280,Atalyja


7.去除评分次数少于50次的音乐和用户。

In [7]:
counts = data['userID'].value_counts() # 计算每个用户的评分次数
data = data[data['userID'].isin(counts[counts >= 50].index)] # 去除评分次数少于50 次的用户

counts = data['name'].value_counts() # 计算每个音乐的评分次数
data = data[data['name'].isin(counts[counts >= 50].index)] # 去除评分次数少于50 次的音乐


In [8]:
# 将音乐和用户ID 转换为数字
name_to_index = {} # 创建 name_to_index 字典
index_to_name = {} # 创建index_to_name 字典
for i, name in enumerate(data['name'].unique()): # 遍历 data 中的name 列
    name_to_index[name] = i # 将name 映射到数字
    index_to_name[i] = name # 将数字映射到name
    
user_id_to_index = {} # 创建user_id_to_index 字典
index_to_user_id = {} # 创建index_to_user_id 字典
for i, user_id in enumerate(data['userID'].unique()): # 遍历data 中的userID 列
    user_id_to_index[user_id] = i # 将user_id 映射到数字
    index_to_user_id[i] = user_id # 将数字映射到user_id
data['name_index'] = data['name'].apply(lambda x: name_to_index[x]) # 将data 中的 name 映射到数字
data['user_index'] = data['userID'].apply(lambda x: user_id_to_index[x]) # 将data 中的 user_id 映射到数字
# # 构建用户-音乐评分矩阵
n_users = len(user_id_to_index) # 获取用户数量
n_artists = len(name_to_index) # 获取音乐数量
ratings_matrix = np.zeros((n_users, n_artists)) # 创建用户-音乐评分矩阵
ratings_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

8.将data中的评分填入用户-音乐评分矩阵，使得ratingsmatrix的输出为。

In [9]:
for row in data.itertuples(): # 遍历data
    ratings_matrix[row[5], row[4]] = row[2] # 将data 中的评分填入用户-音乐评分矩阵
    print(row)
ratings_matrix
#     print(row)
# 计算用户之间的相似度
from sklearn.metrics.pairwise import cosine_similarity # 导入sklearn 库中的余弦相似度计算函数
user_similarity = cosine_similarity(ratings_matrix) # 计算用户之间的相似度
# 为用户推荐音乐
user_index = 0 # 设置用户索引

Pandas(Index=0, userID=2, weight=13883, name='Duran Duran', name_index=0, user_index=0)
Pandas(Index=2, userID=2, weight=11351, name='Air', name_index=1, user_index=0)
Pandas(Index=4, userID=2, weight=8983, name='Kylie Minogue', name_index=2, user_index=0)
Pandas(Index=5, userID=2, weight=6152, name='Daft Punk', name_index=3, user_index=0)
Pandas(Index=7, userID=2, weight=4616, name='Goldfrapp', name_index=4, user_index=0)
Pandas(Index=8, userID=2, weight=4337, name='New Order', name_index=5, user_index=0)
Pandas(Index=13, userID=2, weight=3644, name='Röyksopp', name_index=6, user_index=0)
Pandas(Index=14, userID=2, weight=3579, name='Coldplay', name_index=7, user_index=0)
Pandas(Index=16, userID=2, weight=3301, name='Madonna', name_index=8, user_index=0)
Pandas(Index=19, userID=2, weight=2686, name='Moby', name_index=9, user_index=0)
Pandas(Index=20, userID=2, weight=2654, name='Dido', name_index=10, user_index=0)
Pandas(Index=21, userID=2, weight=2619, name='Depeche Mode', name_index

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



9.获取与用户最相似的用户索引similar users(不包括自己)，使得similar users的输出为:

In [10]:
similar_users = np.argsort(-user_similarity[user_index])[1:] # 获取与用户索引为0 的用户最相似的用户索引
similar_users  
recommended_artists=[]# 创建推荐音乐列表

10.填补以下程序，获取相似用户评分的音乐索引artists_rated。

In [11]:
for i in similar_users: # 遍历相似用户索引
    artists_rated = np.where(ratings_matrix[i] > 0)[0] # 获取相似用户评分的音乐索引
    for j in artists_rated: # 遍历音乐索引
        if ratings_matrix[user_index][j] == 0: #用户未评分
            recommended_artists.append((j, ratings_matrix[i][j])) # 将音乐加入推荐音乐列表
recommended_artists = sorted(recommended_artists, key=lambda x: x[1], reverse=True)[:10] # 筛选出推荐音乐中的前10 首
for artist in recommended_artists: # 遍历推荐音乐
#     print(artists[artists['name'] == index_to_name[artist[0]]]['name'].values[0]) # 打印推荐音乐
    print(index_to_name.get(artist[0]))
#     print(artist)

U2
Blur
Paramore
Evanescence
Christina Aguilera
Shakira
Shakira
Glee Cast
In Flames
All Time Low


In [59]:
artists[artists['name'] == index_to_name[artist[0]]]['name'].values[0]

'Christina Aguilera'